In [1]:
import sys
!{sys.executable} -m pip install spacy
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import spacy

# Testing spaCy

text = """

Dave watched as the forest burned up on the hill,
only a few miles from his house. The car had
been hastily packed and Marta was inside trying to round
up the last of the pets. "Where could she be?" he wondered
as he continued to wait for Marta to appear with the pets.

"""


# NLP constructor
nlp = spacy.load("en_core_web_sm")

print(nlp)

doc = nlp(text)

print(doc)

token_list = [token for token in doc]
print(token_list)



Dave watched as the forest burned up on the hill,
only a few miles from his house. The car had
been hastily packed and Marta was inside trying to round
up the last of the pets. "Where could she be?" he wondered
as he continued to wait for Marta to appear with the pets.

[
, Dave, watched, as, the, forest, burned, up, on, the, hill, ,, 
, only, a, few, miles, from, his, house, ., The, car, had, 
, been, hastily, packed, and, Marta, was, inside, trying, to, round, 
, up, the, last, of, the, pets, ., ", Where, could, she, be, ?, ", he, wondered, 
, as, he, continued, to, wait, for, Marta, to, appear, with, the, pets, ., 
]


In [3]:
# Remove stop words
filtered_tokens = [token for token in doc if not token.is_stop]
print(filtered_tokens)

[
, Dave, watched, forest, burned, hill, ,, 
, miles, house, ., car, 
, hastily, packed, Marta, inside, trying, round, 
, pets, ., ", ?, ", wondered, 
, continued, wait, Marta, appear, pets, ., 
]


In [129]:
# Normalize words

"""

Stemming and lemmatization (only lemmatization is provided by spaCy)
Note the underscore returns the readable version of the lemma here

"""

lemmas = [
    f"Token: {token}, lemma: {token.lemma_}"
    for token in filtered_tokens
]
lemmas

SyntaxError: invalid syntax. Perhaps you forgot a comma? (580949051.py, line 3)

In [5]:
# Vectorize the text
filtered_tokens[1].vector
# Dense arrays have defined values in every space of the array

array([-1.0732636 , -1.5893133 , -0.7485422 ,  0.80338854,  0.199772  ,
        0.00840409,  1.5419112 ,  0.78789294, -0.10507858, -0.08379468,
        1.6370184 ,  0.9981552 , -0.27276078, -0.90784246, -1.248598  ,
       -0.5253062 ,  0.36606142,  0.3220521 ,  0.26947665, -0.6838576 ,
       -1.3466266 ,  0.01122165, -0.24088567, -0.48466757, -0.33174923,
       -0.05325297,  1.8773435 ,  0.5649502 , -0.9605744 ,  0.78610945,
       -0.44939822, -1.4648836 , -0.38066614,  1.0480766 , -0.83412176,
       -0.2217491 , -0.854434  ,  0.35594553, -0.11274697,  1.2787786 ,
       -0.8223142 ,  0.18473107, -0.08983883,  0.6325264 , -1.1029457 ,
        0.37194866,  0.11167954,  1.529881  ,  0.73127055, -0.01238401,
       -0.38741043,  0.24374214,  0.66934144, -0.5147386 , -0.05107623,
       -0.68364084,  1.2553529 , -0.4258138 ,  0.8257121 , -0.40289953,
       -1.0714421 ,  0.8215431 ,  0.1035445 , -0.5627636 ,  0.34108096,
       -0.46954772, -0.6444609 , -0.4248718 , -0.74732184, -0.93

In [22]:
#Load the data
# 80% training data, 20% test data

"""
def load_training_data(
    data_directory: str = "aclImdb/train",
    split: float = 0.8,
    limit: int = 0
) -> tuple:
    reviews = []
    for label in ["pos", "neg"]:
        labeled_directory = f"{data_directory}/{label}"
        for review in os.listdir(labeled_directory):
            if review.endswith(".txt"):
                with open(f"{labeled_directory}/{review}", "r", encoding="utf-8") as f:
                    text = f.read().strip()
                label_id = 1 if label == "pos" else 0
                reviews.append((text, {"cats": {"POSITIVE": label_id, "NEGATIVE": 1 - label_id}}))
    random.shuffle(reviews)
    if limit:
        reviews = reviews[:limit]
    split = int(len(reviews) * split)
    return reviews[:split], reviews[split:]
"""

def load_training_data(data_directory: str = "aclImdb/train", split: float = 0.8, limit: int = 0) -> tuple:
    # Create a list to store the list of tuples
    reviews = [] # Create a list to store the reviews
    # Create two directory paths
    for label in ["pos", "neg"]:
        # Create the full directory path to the pos and neg folders
        labeled_directory = f"{data_directory}/{label}"
        print(labeled_directory)
        # Create the full directory to each of the text files in the pos and negative folders
        # Iterating over all files in the pos and neg folders
        for review in os.listdir(labeled_directory):
            if review.endswith(".txt"):
                with open(f"{labeled_directory}/{review}", "r", encoding="utf-8") as f:
                    text = f.read().strip()
                label_id = 1 if label == "pos" else 0
                reviews.append((text, {"cats": {"POSITIVE": label_id, "NEGATIVE": 1 - label_id}}))
    
    # We created a list of tuples. If we denote a tuple as example, then example[0] is the text, and example[1] is the sentiment
    random.shuffle(reviews)
    # If a limit was passed in, limit the total number of reviews
    # If 0 was passed in, this means there is no limit and this code does not run
    if limit:
        reviews = reviews[:limit]
    split = int(len(reviews) * split)
    # First is the training set, second is the validation/test set
    return reviews[:split], reviews[split:]



In [31]:
# One data entry looks like:
print (train[0:1])

[("This comedy is bound to be good from the get-go. East meets west and east doesn't want to lose...west doesn't know what losing is like. It starts a little slow but it grabs you very soon and it doesn't let go. This is definitely worth seeing.", {'cats': {'POSITIVE': 1, 'NEGATIVE': 0}})]


In [42]:
import os
import random
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding

def train_model(train_data: list, test_data: list, iterations: int = 5) -> None:
    # Initalize the spaCy constructor
    nlp = spacy.blank("en")
    # Ensures that if we update the code, we get the correct "textcat"
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.add_pipe("textcat", last=True)
    else:
        textcat = nlp.get_pipe("textcat")
    textcat.add_label("POSITIVE")
    textcat.add_label("NEGATIVE")

    # Create a listz of excluded pipes. We aren't actually using any other pipes. This is just a double check
    training_excluded_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]

    # Disable all pipeline components not being used
    with nlp.disable_pipes(training_excluded_pipes):
        optimizer = nlp.begin_training()
        print("Start of training:")
        print("Loss\t\tPrecision\t\tRecall\t\tF-score")
        # Starting batch size, maximum batch size, rate at which batch size grows in each iteration
        # This allows the model to learn from a variety of batch sizes, potentially increasing accuracy
        batch_sizes = compounding(4, 32.0, 1.001)
        for i in range(iterations):
            loss = {}
            random.shuffle(train_data)
            batches = minibatch(train_data, size=batch_sizes)
            for batch in batches:
                examples = []
                for text, label in batch:
                    doc = nlp.make_doc(text)
                    examples.append(Example.from_dict(doc, label))
                nlp.update(examples, losses=loss)
                
            # Evaluate the model
            evaluation_results = evaluate_model(nlp.tokenizer, textcat, test_data)
            print(
                f"{loss['textcat']}\t{evaluation_results['precision']}"
                f"\t{evaluation_results['recall']}\t{evaluation_results['f-score']}"
            )
    # Save the model
    with nlp.use_params(optimizer.averages):
        nlp.to_disk("model_artifacts")

In [40]:
# Evaluate the progress of training
# True positive, true negative, false positive, false negative
# Calculate the precision (# of successful classification vs # of incorrect)
# Calculate the recall (# of sucessful classification / # of incorrect)
# F-score - metric for evaluating effectiveness of a binary classification model

def evaluate_model(tokenizer, textcat, test_data: list) -> dict:
    true_positives = 0
    false_positives = 1e-8
    true_negatives = 0
    false_negatives = 1e-8
    for text, annotation in test_data:
        doc = tokenizer(text)
        truth = annotation["cats"]  # True labels
        predicted = textcat(doc)    # Predicted labels and scores
        
        for label, score in predicted.cats.items():
            if label not in truth:
                continue  # Skip if the label is not in the true labels
            
            # Check if the predicted score meets a threshold (e.g., 0.5) to classify as positive or negative
            if score >= 0.5:
                # Predicted as positive
                if truth[label] == 1:
                    # True positive
                    true_positives += 1
                else:
                    # False positive
                    false_positives += 1
            else:
                # Predicted as negative
                if truth[label] == 0:
                    # True negative
                    true_negatives += 1
                else:
                    # False negative
                    false_negatives += 1

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    if precision + recall == 0:
        f_score = 0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    
    return {"precision": precision, "recall": recall, "f-score": f_score}

In [80]:
# train and test, are lists of tuples
# The limit parameter limits the number of training and testing examples
train, test = load_training_data(limit=2500)

aclImdb/train/pos
aclImdb/train/neg


In [81]:
# Train the model
train_model(train, test)

Start of training:
Loss		Precision		Recall		F-score
91.11163692548871	0.78199999998436	0.78199999998436	0.7819999999843599
27.476687906397274	0.7599999999848	0.7599999999848	0.7599999999848
7.321709191841013	0.76799999998464	0.76799999998464	0.76799999998464
1.2663690280867284	0.80399999998392	0.80399999998392	0.8039999999839199
0.05400360797214075	0.79599999998408	0.79599999998408	0.79599999998408


In [128]:
# Load the saved model
nlp = spacy.load("model_artifacts")

from string import punctuation
#load spacy's default language

# Test review
TEST_REVIEW = """
Friendly and efficient customer service. 
The store owner was really patient. 
My new glasses arrived in less than a week and they 
made sure every details and aspects work perfectly for me. 
Overall, I'm really satisfied with it.
This store also gives reasonable price compared to others. 
I'd recommend trying this store out!
"""

# Process the test review
doc = nlp(TEST_REVIEW)

# Get the predicted category
predicted_category = doc.cats
print("Predicted category probabilities:")
print(predicted_category)

Predicted category probabilities:
{'POSITIVE': 0.8926675915718079, 'NEGATIVE': 0.10733240097761154}


In [130]:
# Keyword extraction
# Using Counter, extract the top five nouns from all reviews combined
# Here, we are only using one review

from collections import Counter

nlp = spacy.load("en_core_web_sm")

doc = nlp(TEST_REVIEW)

nouns = []
adjectives = []

# Define functions
def is_token_allowed(token):
    # Filter tokens that are NOT punctuation or stop words
    return bool(token and str(token).strip() and not token.is_stop and not token.is_punct)

def preprocess_token(token):
    # Lowercase, and remove whitespace
    return token.lemma_.strip().lower()

complete_filtered_tokens = Counter([preprocess_token(token) for token in doc if is_token_allowed(token)])

nouns = []
adjectives = []
for token in doc:
    if token.pos_ == "NOUN":
        nouns.append(token)
    if token.pos_ == "ADJ":
        adjectives.append(token)


top_nouns = Counter(nouns).most_common(5)

print(top_nouns)

[(customer, 1), (service, 1), (store, 1), (owner, 1), (glasses, 1)]
